In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from define_dataset import define_dataset
import time
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [6]:
dataset = define_dataset()
tr_set = dataset.tr_set
test_set = dataset.test_set
results_dict = dataset.results_dict
num_bins_arr = [40, 60, 100]
ROI_threshold_arr = [0.85, 0.9, 0.95]
best_clf_score = 0

In [9]:
for ROI_threshold in ROI_threshold_arr:
    for num_bins in num_bins_arr:
        print('Num bins:', num_bins, 'ROI threshold:', ROI_threshold)

        X = np.load('bins/bins_arr_'+str(num_bins)+'r'+str(ROI_threshold)+'.npy')
        y = np.zeros(np.shape(tr_set), dtype='int')
        for i, slide_name in enumerate(tr_set):
            y[i] = results_dict[slide_name]

        clf = DecisionTreeClassifier()
        scores = cross_val_score(clf, X, y, cv=6)
        score = sum(scores)/len(scores)
        print(score)
        #retrain
        clf.fit(X, y)

        if score >= best_clf_score:
            best_clf_score = score
            best_clf = clf
            best_bins = num_bins
            best_ROI = ROI_threshold

Num bins: 40 ROI threshold: 0.85
0.75
Num bins: 60 ROI threshold: 0.85
0.638888888888889
Num bins: 100 ROI threshold: 0.85
0.7222222222222223
Num bins: 40 ROI threshold: 0.9
0.7777777777777778
Num bins: 60 ROI threshold: 0.9
0.7777777777777778
Num bins: 100 ROI threshold: 0.9
0.7222222222222223
Num bins: 40 ROI threshold: 0.95
0.75
Num bins: 60 ROI threshold: 0.95
0.7222222222222223
Num bins: 100 ROI threshold: 0.95
0.7222222222222223


Test set

In [12]:
print('best bins:', best_bins, ', best ROI threshold:', best_ROI, ', best score:', best_clf_score)
X_test = np.load('./bins/bins_arr_test_'+str(best_bins)+'r'+str(best_ROI)+'.npy')
y_test = np.zeros(np.shape(test_set), dtype='int')
for i, slide_name in enumerate(test_set):
    y_test[i] = results_dict[slide_name]

y_pred = best_clf.predict(X_test)

print('predictions:',y_pred)
print('real y:', y_test)
score = accuracy_score(y_test, y_pred)
errors = np.size(test_set) - accuracy_score(y_test, y_pred, normalize=False)
print('Test result:', score)
print('Errors:', errors)

best bins: 40 , best ROI threshold: 0.95 , best score: 0.7833333333333333
predictions: [1 1 0 1 1 1]
real y: [0 0 0 1 1 1]
Test result: 0.6666666666666666
Errors: 2
